In [ ]:
"""
📌 Project Details:

- Subject: Programming for Artificial Intelligence (AI271)
- Instructor: Maryam Afzal
- Section: A6
- Group Members: MUIZZ KHAN (F2023376059) & ALIZA ZAINAB (F2023376320)
- Topic: Fake News Articles Detection
"""

'\n📌 Project Details:\n\n- Subject: Programming for Artificial Intelligence (AI271)\n- Instructor: Maryam Afzal\n- Section: A6\n- Group Members: MUIZZ KHAN (F2023376059) & ALIZA ZAINAB (F2023376320)\n- Topic: Fake News Articles Detection\n'

Dataset


* id: unique id
*   title: title of the news article
* author: author of the news article
* text: text part of the article
* labels: a label that marks whether an article is real or fake:
          1: Fake
          0: Real

Importing Dependencies

In [ ]:
import numpy as np   #for making numpy arrays
import pandas as pd  #for creating and storing dataframes
import re            #for searching words in a text
from nltk.corpus import stopwords     #removes words in dataset
from nltk.stem.porter import PorterStemmer  #for stemming words
from sklearn.feature_extraction.text import TfidfVectorizer   #converts text into feature vectors
from sklearn.model_selection import train_test_split      #splits data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC  # Support Vector Machine
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data pre-processing

In [ ]:
#loading dataset into pandas dataframe
news_dataset = pd.read_csv('/content/train (1).csv',encoding='latin-1')

<ipython-input-78-392b6cc5fef0>:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211) have mixed types. Specify dtype option on import or set low_memory=False.
  news_dataset = pd.read_csv('/content/train (1).csv',encoding='latin-1')


In [ ]:
news_dataset.shape #rows and columns of dataset

(5746, 213)

In [ ]:
#printing the first 5 rows
news_dataset.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 203,Unnamed: 204,Unnamed: 205,Unnamed: 206,Unnamed: 207,Unnamed: 208,Unnamed: 209,Unnamed: 210,Unnamed: 211,Unnamed: 212
0,0,House Dem Aide: We Didnât Even See Comeyâs...,Darrell Lucus,House Dem Aide: We Didnât Even See Comeyâs...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#counting number of missing values
news_dataset.isnull().sum()

,0
id,4715
title,4754
author,4847
text,4793
label,4834
...,...
Unnamed: 208,5745
Unnamed: 209,5745
Unnamed: 210,5745
Unnamed: 211,5745


In [ ]:
#replacing null values with empty string
news_dataset = news_dataset.fillna('')

In [ ]:
#merging the author name and news title
news_dataset['content'] = news_dataset['author'] +' ' + news_dataset['title']

In [ ]:
print(news_dataset['content'])

0       Darrell Lucus House Dem Aide: We Didnât Even...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
5741                                                     
5742                                                     
5743                                                     
5744                                                     
5745                                                     
Name: content, Length: 5746, dtype: object


In [ ]:
#separating the data & label
X = news_dataset.drop(columns='label',axis=1)   #removing the column so using 1
Y = news_dataset['label']

In [ ]:
print(X)
print(Y)

     id                                              title  \
0     0  House Dem Aide: We Didnât Even See Comeyâs...   
1     1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2     2                  Why the Truth Might Get You Fired   
3     3  15 Civilians Killed In Single US Airstrike Hav...   
4     4  Iranian woman jailed for fictional unpublished...   
...  ..                                                ...   
5741                                                         
5742                                                         
5743                                                         
5744                                                         
5745                                                         

                  author                                               text  \
0          Darrell Lucus  House Dem Aide: We Didnât Even See Comeyâs...   
1        Daniel J. Flynn  Ever get the feeling your life circles the rou...   
2     Consortiumne

Stemming

reducing a word to its root word

In [ ]:
port_stem = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content) #substitutes certain values(numbers,punctuations etc)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
news_dataset['content'] = news_dataset['content'].apply(stemming) #use stemming function on content

In [ ]:
print(news_dataset['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
5741                                                     
5742                                                     
5743                                                     
5744                                                     
5745                                                     
Name: content, Length: 5746, dtype: object


In [ ]:
#separating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ... '' '' '']


In [ ]:
print(Y)

['1' '0' '1' ... '' '' '']


In [ ]:
Y.shape

(5746,)

In [ ]:
#converting actual data into numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

x = vectorizer.transform(X)

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ... '' '' '']


Splitting the dataset to training & test data

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x,Y,test_size=0.2,random_state=2)

Training the model: Logistic Regression

In [ ]:
model = LogisticRegression()

In [ ]:
model_params = {'C': [0.01, 0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']} #hyperparameters for best performance
model_grid = GridSearchCV(model, model_params, cv=5)

In [ ]:
model_grid.fit(X_train, Y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5, estimator=LogisticRegression(),
             param_grid={'C': [0.01, 0.1, 1, 10],
                         'solver': ['liblinear', 'lbfgs']})

In [ ]:
model_best = model_grid.best_estimator_

Evaluation(using Logistic Regression)

accuracy score


In [ ]:
#accuracy score on the training data
X_train_accuracy = accuracy_score(Y_train, model_best.predict(X_train))

In [ ]:
print('Accuracy Score of the training data: ',X_train_accuracy)

Accuracy Score of the training data:  0.9989120974760661


In [ ]:
#accuracy score on the test data
X_test_accuracy = accuracy_score(Y_test, model_best.predict(X_test))

In [ ]:
print('Accuracy Score of the test data: ', X_test_accuracy)

Accuracy Score of the test data:  0.9260869565217391


SVM

In [ ]:
# Train SVM model
svm_model= SVC()
svm_params = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}
svm_grid = GridSearchCV(svm_model, svm_params, cv=5)

In [ ]:
svm_grid.fit(X_train, Y_train)
svm_best = svm_grid.best_estimator_

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Evaluation(using SVM)

In [ ]:
#accuracy on training data
svm_train_accuracy = accuracy_score(Y_train, svm_best.predict(X_train))
print("Accuracy score on taining data", svm_train_accuracy)

Accuracy score on taining data 0.9989120974760661


In [ ]:
# Accuracy on test data
svm_test_accuracy = accuracy_score(Y_test, svm_best.predict(X_test))
print("Accuracy on test data", svm_test_accuracy)


Accuracy on test data 0.9391304347826087


Making a Predictive System

In [ ]:
#using Logistic regression
X_new = X_test[0]


In [ ]:
prediction = model_best.predict(X_new)
print(prediction)

['']


In [ ]:
if (prediction[0] == 0):
  print('The news is Real')
else:
  print('The news is Fake')

The news is Fake


In [ ]:
print(Y_test[0])

In [ ]:
X_new = X_test[1]  # Pick another sample for prediction

# Reshape it to match SVM input
X_new_reshaped = X_new.reshape(1, -1)

# Make prediction using SVM
prediction_svm = svm_best.predict(X_new_reshaped)
print(prediction_svm)

if prediction_svm[0] == 0:
    print("The news is Real")
else:
    print("The news is Fake")


['']
The news is Fake


In [ ]:
print(Y_test[1])

Comparison of 2 models

In [ ]:
#classification report for logictic regression
lr_prediction = model_best.predict(X_test)
lr_report = classification_report(Y_test, lr_prediction, output_dict=True)
lr_df = pd.DataFrame(lr_report).transpose()
print("Logistic Regression Classification Report:")
print(lr_df)


Logistic Regression Classification Report:
                                                    precision    recall  \
                                                     0.921606  0.997930   
 Allonâs men distributed fliers                    0.000000  0.000000   
 British Foreign Secretary Lord Arthur James Ba...   0.000000  0.000000   
 Vietnam and other countries in the region. The...   0.000000  0.000000   
 Yigal Allon. [91] On May 1                          0.000000  0.000000   
 all parties have agreed that it can be done         0.000000  0.000000   
 and Haifa. Most of the violence was initiated ...   0.000000  0.000000   
 and for the most part the Arabs simply fled.â...   0.000000  0.000000   
 and we and our American partners would target ...   0.000000  0.000000   
 by his own account                                  0.000000  0.000000   
 creating faits accomplis became part of the ov...   0.000000  0.000000   
 even if past the deadline. So one would think ...   0.00

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# SVM report
svm_prediction = svm_best.predict(X_test)
svm_report = classification_report(Y_test, svm_prediction, output_dict=True)
svm_df = pd.DataFrame(svm_report).transpose()
print("SVM Classification Report:")
print(svm_df)

SVM Classification Report:
                                                    precision    recall  \
                                                     0.935078  0.998965   
 Allonâs men distributed fliers                    0.000000  0.000000   
 British Foreign Secretary Lord Arthur James Ba...   0.000000  0.000000   
 Vietnam and other countries in the region. The...   0.000000  0.000000   
 Yigal Allon. [91] On May 1                          0.000000  0.000000   
 all parties have agreed that it can be done         0.000000  0.000000   
 and Haifa. Most of the violence was initiated ...   0.000000  0.000000   
 and for the most part the Arabs simply fled.â...   0.000000  0.000000   
 and we and our American partners would target ...   0.000000  0.000000   
 by his own account                                  0.000000  0.000000   
 creating faits accomplis became part of the ov...   0.000000  0.000000   
 even if past the deadline. So one would think ...   0.000000  0.000000  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
print("Accuracy Comparison:")
print(f"Logistic Regression Accuracy: {test_data_accuracy:.4f}")
print(f"SVM Accuracy: {svm_accuracy:.4f}")

Accuracy Comparison:
Logistic Regression Accuracy: 0.9035
SVM Accuracy: 0.9348
